In [ ]:
import cv2
import os
import json
import qrcode
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageTk
from datetime import datetime
from pathlib import Path
import logging
import customtkinter as ctk
import threading
import time
import re

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Core Logic ---
class IDCardController:
    """
    Handles the backend logic for student data, face capture, and ID card generation.
    """
    def __init__(self):
        self.FACE_DB_PATH = Path("student_faces")
        self.PHOTO_DIR = Path("card_photos")
        self.OUTPUT_DIR = Path("id_cards")
        self.STUDENT_DATA_FILE = Path("student_database.json")
        
        for directory in [self.FACE_DB_PATH, self.PHOTO_DIR, self.OUTPUT_DIR]:
            directory.mkdir(exist_ok=True)
            
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        self.student_data = self.load_student_database()
        
        try:
            self.font = ImageFont.truetype("arial.ttf", 20)
            self.font_large = ImageFont.truetype("arial.ttf", 24)
        except OSError:
            logger.warning("Arial font not found. Using default font.")
            self.font = ImageFont.load_default()
            self.font_large = ImageFont.load_default()

    def load_student_database(self):
        try:
            if self.STUDENT_DATA_FILE.exists():
                with open(self.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    logger.info(f"Loaded {len(data)} students from database.")
                    return data
            else:
                logger.info("Created new student database.")
                return {}
        except Exception as e:
            logger.error(f"Error loading database: {e}")
            return {}

    def save_student_database(self):
        try:
            with open(self.STUDENT_DATA_FILE, 'w', encoding='utf-8') as f:
                json.dump(self.student_data, f, indent=4, ensure_ascii=False)
            logger.info("Student database saved successfully.")
        except Exception as e:
            logger.error(f"Error saving database: {e}")

    def get_next_student_id(self):
        """Generates the next student ID in the format S001, S002, etc."""
        if not self.student_data:
            return "S001"
        
        max_num = 0
        for student_id in self.student_data.keys():
            if student_id.startswith('S') and student_id[1:].isdigit():
                num = int(student_id[1:])
                if num > max_num:
                    max_num = num
        
        next_num = max_num + 1
        return f"S{next_num:03d}"

    def generate_qr_code(self, student_data):
        """Generate QR code with only the student ID for reliability."""
        qr_data = student_data['id']
        qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=4)
        qr.add_data(qr_data)
        qr.make(fit=True)
        qr_img = qr.make_image(fill_color="black", back_color="white").resize((120, 120))
        return qr_img

    def create_id_card(self, student_info):
        logger.info(f"Generating ID card for {student_info['name']}...")
        card_width, card_height = 400, 280
        card = Image.new("RGB", (card_width, card_height), "white")
        draw = ImageDraw.Draw(card)
        
        draw.rectangle([(0, 0), (card_width, 50)], fill="#003366")
        draw.text((15, 15), "STUDENT ID CARD", fill="white", font=self.font_large)
        draw.rectangle([(5, 5), (card_width - 5, card_height - 5)], outline="#003366", width=2)

        photo_path = self.PHOTO_DIR / f"{student_info['id']}.jpg"
        if photo_path.exists():
            try:
                photo = Image.open(photo_path).resize((90, 110), Image.Resampling.LANCZOS)
                card.paste(photo, (280, 65))
                draw.rectangle([(279, 64), (371, 176)], outline="gray", width=1)
            except Exception as e:
                logger.error(f"Could not process photo for {student_info['id']}: {e}")
        else:
            draw.rectangle([(280, 65), (370, 175)], fill="#E0E0E0", outline="gray")
            draw.text((295, 110), "NO\nPHOTO", fill="black", font=self.font)

        y_pos = 60
        draw.text((20, y_pos), f"ID: {student_info['id']}", fill="black", font=self.font)
        y_pos += 25
        draw.text((20, y_pos), f"Name: {student_info['name']}", fill="black", font=self.font)
        y_pos += 25
        draw.text((20, y_pos), f"Program: {student_info['program']}", fill="black", font=self.font)
        y_pos += 24
        draw.text((20, y_pos), f"Batch: {student_info['batch']}", fill="black", font=self.font)

        qr_code = self.generate_qr_code(student_info)
        card.paste(qr_code, (20, 153))
        
        issue_date = datetime.now().strftime("%Y-%m-%d")
        draw.text((280, 190), f"Issued: {issue_date}", fill="gray", font=ImageFont.load_default())
        
        card_path = self.OUTPUT_DIR / f"{student_info['id']}_id_card.png"
        card.save(card_path)
        logger.info(f"ID card saved to {card_path}")
        return card_path




In [ ]:
# --- GUI Application ---
class App(ctk.CTk):
    # Constants for auto-capture
    VERIFICATION_IMAGE_COUNT = 30
    AUTO_CAPTURE_DELAY = 0.25

    def __init__(self, controller):
        super().__init__()
        self.controller = controller

        # --- UI IMPROVEMENT ---: More descriptive title and larger default size
        self.title("Student ID Card Generation System")
        self.geometry("1280x720") 
        ctk.set_appearance_mode("System")
        ctk.set_default_color_theme("blue")

        # --- UI IMPROVEMENT ---: Configure grid weights for responsive resizing
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # Registration flow state
        self.registration_step = 0
        self.current_student_info = {}

        # --- Sidebar ---
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)
        
        # --- UI IMPROVEMENT ---: Larger font for the menu title
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Menu", font=ctk.CTkFont(size=24, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        self.btn_register = ctk.CTkButton(self.sidebar_frame, text="Register Student", command=self.show_register_frame)
        self.btn_register.grid(row=1, column=0, padx=20, pady=10)

        self.btn_list = ctk.CTkButton(self.sidebar_frame, text="List Students", command=self.show_list_frame)
        self.btn_list.grid(row=2, column=0, padx=20, pady=10)

        self.btn_exit = ctk.CTkButton(self.sidebar_frame, text="Exit", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=self.on_closing)
        self.btn_exit.grid(row=6, column=0, padx=20, pady=20, sticky="s")

        # --- Frame Initialization ---
        self.register_frame = self.create_register_frame()
        self.list_frame = self.create_list_frame()

        # --- Camera State ---
        self.cap = None
        self.is_capturing = False
        self.capture_mode = None
        self.captured_count = 0
        self.last_capture_time = 0
        
        self.update_next_student_id()
        self.show_register_frame()

    def create_register_frame(self):
        frame = ctk.CTkFrame(self)
        # --- UI IMPROVEMENT ---: Grid weights for responsive layout
        frame.grid_columnconfigure(0, weight=1) # Form side
        frame.grid_columnconfigure(1, weight=1) # Camera side
        frame.grid_rowconfigure(0, weight=1)

        # Left side - Form and controls
        left_frame = ctk.CTkFrame(frame)
        left_frame.grid(row=0, column=0, padx=(20, 10), pady=20, sticky="nsew")
        left_frame.grid_rowconfigure(1, weight=1) # Make the form container expand
        
        # Progress indicator
        self.progress_frame = ctk.CTkFrame(left_frame)
        self.progress_frame.grid(row=0, column=0, padx=20, pady=(20, 10), sticky="ew")
        self.progress_frame.grid_columnconfigure((0,1,2,3), weight=1) # Distribute labels evenly
        
        self.step_labels = []
        steps = ["1. Details", "2. Verification", "3. ID Photo", "4. Complete"]
        for i, step in enumerate(steps):
            label = ctk.CTkLabel(self.progress_frame, text=step, font=ctk.CTkFont(size=12))
            label.grid(row=0, column=i, padx=5, pady=10)
            self.step_labels.append(label)

        # Form container
        self.form_container = ctk.CTkFrame(left_frame)
        self.form_container.grid(row=1, column=0, padx=20, pady=10, sticky="nsew")
        
        self.create_form_step()

        # Right side - Camera and ID card display
        right_frame = ctk.CTkFrame(frame)
        right_frame.grid(row=0, column=1, padx=(10, 20), pady=20, sticky="nsew")
        right_frame.grid_rowconfigure(0, weight=1) # Make display area expand
        right_frame.grid_columnconfigure(0, weight=1)
        
        self.display_frame = ctk.CTkFrame(right_frame)
        self.display_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        self.display_frame.grid_propagate(False) # Prevent children from resizing the frame
        
        self.video_label = ctk.CTkLabel(self.display_frame, text="")
        self.video_label.pack(expand=True, fill="both", padx=10, pady=10)
        
        self.status_label = ctk.CTkLabel(right_frame, text="Status: Ready to start registration", anchor="w")
        self.status_label.grid(row=1, column=0, padx=20, pady=(0, 20), sticky="ew")

        return frame

    def create_form_step(self):
        for widget in self.form_container.winfo_children():
            widget.destroy()
            
        # --- UI IMPROVEMENT ---: Use grid layout for the form for better alignment and responsiveness
        self.form_container.grid_columnconfigure(1, weight=1) # Allow entry column to expand

        # Title
        ctk.CTkLabel(self.form_container, text="Student Information", font=ctk.CTkFont(size=20, weight="bold")).grid(
            row=0, column=0, columnspan=2, pady=(10, 20))

        # Student ID
        ctk.CTkLabel(self.form_container, text="Student ID:").grid(row=1, column=0, padx=10, pady=10, sticky="w")
        self.next_id_var = ctk.StringVar()
        self.entry_id = ctk.CTkEntry(self.form_container, textvariable=self.next_id_var, state="readonly")
        self.entry_id.grid(row=1, column=1, padx=10, pady=10, sticky="ew")

        # Name
        ctk.CTkLabel(self.form_container, text="Full Name:").grid(row=2, column=0, padx=10, pady=10, sticky="w")
        self.entry_name = ctk.CTkEntry(self.form_container, placeholder_text="e.g., John Doe")
        self.entry_name.grid(row=2, column=1, padx=10, pady=10, sticky="ew")

        # Program
        ctk.CTkLabel(self.form_container, text="Program:").grid(row=3, column=0, padx=10, pady=10, sticky="w")
        self.entry_program = ctk.CTkEntry(self.form_container, placeholder_text="e.g., Computer Science")
        self.entry_program.grid(row=3, column=1, padx=10, pady=10, sticky="ew")

        # CGPA
        ctk.CTkLabel(self.form_container, text="CGPA:").grid(row=4, column=0, padx=10, pady=10, sticky="w")
        self.entry_cgpa = ctk.CTkEntry(self.form_container, placeholder_text="e.g., 4.00")
        self.entry_cgpa.grid(row=4, column=1, padx=10, pady=10, sticky="ew")

        # Email
        ctk.CTkLabel(self.form_container, text="Email:").grid(row=5, column=0, padx=10, pady=10, sticky="w")
        self.entry_email = ctk.CTkEntry(self.form_container, placeholder_text="e.g., abc_wm22@student.tarc.edu.my")
        self.entry_email.grid(row=5, column=1, padx=10, pady=10, sticky="ew")

        # Phone
        ctk.CTkLabel(self.form_container, text="Phone (Optional):").grid(row=6, column=0, padx=10, pady=10, sticky="w")
        self.entry_phone = ctk.CTkEntry(self.form_container, placeholder_text="e.g., 012-3456789")
        self.entry_phone.grid(row=6, column=1, padx=10, pady=10, sticky="ew")

        # Next button
        ctk.CTkButton(self.form_container, text="Next: Start Verification Capture", command=self.proceed_to_verification, fg_color="green").grid(
            row=7, column=0, columnspan=2, pady=(30, 10))
            
        self.update_next_student_id()

    def create_verification_step(self):
        for widget in self.form_container.winfo_children():
            widget.destroy()

        ctk.CTkLabel(self.form_container, text="Verification Photos", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=(10, 20))
        ctk.CTkLabel(self.form_container, text="Please look at the camera.\nWe will automatically capture 30 photos for verification.", 
                       wraplength=350, justify="center", font=ctk.CTkFont(size=14)).pack(pady=10)
        
        self.progress_label = ctk.CTkLabel(self.form_container, text="Progress: 0/30 captured", font=ctk.CTkFont(size=14))
        self.progress_label.pack(pady=10)

        self.progress_bar = ctk.CTkProgressBar(self.form_container, width=300)
        self.progress_bar.set(0)
        self.progress_bar.pack(pady=10)

        button_frame = ctk.CTkFrame(self.form_container, fg_color="transparent")
        button_frame.pack(pady=20)
        
        self.start_verification_btn = ctk.CTkButton(button_frame, text="Start Capture", command=self.start_verification_capture)
        self.start_verification_btn.pack(side="left", padx=5)
        ctk.CTkButton(button_frame, text="Skip to Photo", command=self.proceed_to_photo).pack(side="left", padx=5)

    def create_photo_step(self):
        for widget in self.form_container.winfo_children():
            widget.destroy()

        ctk.CTkLabel(self.form_container, text="ID Photo Capture", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=(10, 20))
        ctk.CTkLabel(self.form_container, text="Please position your face in the center of the camera view.\nWe will automatically capture your ID photo.",
                       wraplength=350, justify="center", font=ctk.CTkFont(size=14)).pack(pady=10)
        
        button_frame = ctk.CTkFrame(self.form_container, fg_color="transparent")
        button_frame.pack(pady=20)
        
        ctk.CTkButton(button_frame, text="Start ID Photo Capture", command=self.start_id_photo_capture).pack(side="left", padx=5)
    
    def create_complete_step(self):
        for widget in self.form_container.winfo_children():
            widget.destroy()

        ctk.CTkLabel(self.form_container, text="✅ Registration Complete!", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=(10, 20))
        
        if self.current_student_info:
            info_text = f"Student: {self.current_student_info['name']}\nID: {self.current_student_info['id']}\nProgram: {self.current_student_info['program']}\nCGPA: {self.current_student_info.get('cgpa', 'N/A')}\nAcademic Performance: {self.current_student_info.get('academic_performance', 'N/A')}\nBatch: {self.current_student_info.get('batch', 'N/A')}"
            ctk.CTkLabel(self.form_container, text=info_text, font=ctk.CTkFont(size=14), justify="left").pack(pady=10)

        ctk.CTkButton(self.form_container, text="Register Another Student", command=self.reset_registration, fg_color="blue").pack(pady=10)
        ctk.CTkButton(self.form_container, text="View Student List", command=self.show_list_frame).pack(pady=5)
    
    def update_progress_display(self):
        for i, label in enumerate(self.step_labels):
            if i == self.registration_step:
                label.configure(text_color=("#1F6AA5", "#4A90E2"), font=ctk.CTkFont(size=12, weight="bold"))
            elif i < self.registration_step:
                label.configure(text_color="green", font=ctk.CTkFont(size=12))
            else:
                label.configure(text_color=("gray50", "gray40"), font=ctk.CTkFont(size=12))

    def generate_academic_performance(self):
        cgpa = self.entry_cgpa.get().strip()
        try:
            cgpa_val = float(cgpa)
            if cgpa_val >= 3.75:
                return "First Class"
            elif cgpa_val >= 3.25:
                return "Second Class Upper"
            elif cgpa_val >= 2.75:
                return "Second Class Lower"
            elif cgpa_val >= 2.00:
                return "Pass"
            else:
                return "Fail"
        except ValueError:
            return "N/A"
            
    def generate_batch(self):
        email = self.entry_email.get().strip()
        m = re.search(r'[A-Za-z]+-[A-Za-z]+(\d+)\@student\.tarc\.edu\.my$', email)
        return m.group(1) if m else "N/A"

    def proceed_to_verification(self):
        pattern = r"^[a-z]+-[a-z]+\d{2,}@student\.tarc\.edu\.my$"
        
        name = self.entry_name.get().strip()
        program = self.entry_program.get().strip()
        email = self.entry_email.get().strip()
        cgpa = self.entry_cgpa.get().strip()
        #name
        if not name:
            self.status_label.configure(text="Error: Please enter the student's full name.", text_color="red")
            self.entry_name.focus_set()
            return
        #program
        if not program:
            self.status_label.configure(text="Error: Please enter the student's program.", text_color="red")
            self.entry_program.focus_set()
            return
        #cgpa
        if not cgpa:
            self.status_label.configure(text="Error: Please enter the student's cgpa.", text_color="red")
            self.entry_cgpa.focus_set()
            return
        try:
            cgpa_val = float(cgpa)
        except ValueError:
            self.status_label.configure(text="CGPA must be a number (e.g., 3.75).", text_color="red")
            self.entry_cgpa.focus_set()
            return

        if not (2.0 <= cgpa_val <= 4.0):
            self.status_label.configure(text="CGPA must be between 2.00 and 4.00.", text_color="red")
            self.entry_cgpa.focus_set()
            return
        #email
        if not email:
            self.status_label.configure(text="Error: Please enter the student's email.", text_color="red")
            self.entry_email.focus_set()
            return
        elif not re.match(pattern, email):
            self.status_label.configure(
                text="Email must be like laiyl-wm22@student.tarc.edu.my",
                text_color="red"
            )
            self.entry_email.focus_set()
            return
        

        self.current_student_info = {
            'id': self.next_id_var.get(), 'name': name,
            'program': self.entry_program.get().strip() or "N/A",
            'email': self.entry_email.get().strip(), 'phone': self.entry_phone.get().strip(),
            'cgpa': self.entry_cgpa.get().strip(),
            'academic_performance': self.generate_academic_performance(),
            'batch': self.generate_batch(),
            'created_date': datetime.now().isoformat()
        }
        (self.controller.FACE_DB_PATH / self.current_student_info['id']).mkdir(exist_ok=True)
        self.registration_step = 1
        self.update_progress_display()
        self.create_verification_step()
        self.status_label.configure(text="Ready for verification photo capture.", text_color=("black", "white"))

    def proceed_to_photo(self):
        self.stop_video_feed()
        self.registration_step = 2
        self.update_progress_display()
        self.create_photo_step()
        self.status_label.configure(text="Ready for ID photo capture.", text_color=("black", "white"))

    def complete_registration(self):
        self.stop_video_feed()
        self.controller.student_data[self.current_student_info['id']] = self.current_student_info
        self.controller.save_student_database()
        card_path = self.controller.create_id_card(self.current_student_info)
        self.display_id_card(card_path)
        self.registration_step = 3
        self.update_progress_display()
        self.create_complete_step()
        self.status_label.configure(text="Registration completed successfully!", text_color="green")
        
    def display_id_card(self, card_path):
        try:
            card_img = Image.open(card_path)
            # --- UI IMPROVEMENT ---: Resize based on the available space in the display frame
            frame_w = self.display_frame.winfo_width()
            frame_h = self.display_frame.winfo_height()
            
            # Maintain aspect ratio while fitting into the frame
            img_aspect = card_img.width / card_img.height
            frame_aspect = frame_w / frame_h
            
            if img_aspect > frame_aspect:
                # Image is wider than frame, fit to width
                new_w = frame_w - 20 # padding
                new_h = int(new_w / img_aspect)
            else:
                # Image is taller than frame, fit to height
                new_h = frame_h - 20 # padding
                new_w = int(new_h * img_aspect)

            card_img = card_img.resize((new_w, new_h), Image.Resampling.LANCZOS)
            
            card_tk = ImageTk.PhotoImage(card_img)
            self.video_label.configure(image=card_tk, text="")
            self.video_label.image = card_tk
        except Exception as e:
            logger.error(f"Error displaying ID card: {e}")
            self.video_label.configure(image=None, text="ID Card Generated\n(Display error)")

    def reset_registration(self):
        self.registration_step = 0
        self.current_student_info = {}
        self.captured_count = 0
        self.stop_video_feed()
        self.update_progress_display()
        self.create_form_step()
        self.video_label.configure(image=None, text="")
        self.status_label.configure(text="Status: Ready to start registration", text_color=("black", "white"))
    
    def create_list_frame(self):
        frame = ctk.CTkFrame(self)
        # --- UI IMPROVEMENT ---: Give more weight to the list than the preview
        frame.grid_columnconfigure(0, weight=3)
        frame.grid_columnconfigure(1, weight=2)
        frame.grid_rowconfigure(0, weight=1)

        # Left side - Student list
        list_container = ctk.CTkFrame(frame)
        list_container.grid(row=0, column=0, padx=(20, 10), pady=20, sticky="nsew")
        list_container.grid_rowconfigure(2, weight=1) # Make scrollable frame expand
        list_container.grid_columnconfigure(0, weight=1)

        header_frame = ctk.CTkFrame(list_container, fg_color="transparent")
        header_frame.grid(row=0, column=0, sticky="ew", padx=20)
        header_frame.grid_columnconfigure(0, weight=1)

        ctk.CTkLabel(header_frame, text="Student Database", font=ctk.CTkFont(size=20, weight="bold")).grid(row=0, column=0, pady=(10,0), sticky="w")
        
        # --- UI IMPROVEMENT ---: Added a refresh button
        ctk.CTkButton(header_frame, text="🔄 Refresh", width=100, command=self.populate_student_list).grid(row=0, column=1, pady=(10,0), sticky="e")
        
        self.scrollable_frame = ctk.CTkScrollableFrame(list_container, label_text="All Students")
        self.scrollable_frame.grid(row=2, column=0, padx=20, pady=(0, 20), sticky="nsew")
        
        # --- UI IMPROVEMENT ---: Assign different weights to columns for a better layout
        self.scrollable_frame.grid_columnconfigure(0, weight=1) # ID
        self.scrollable_frame.grid_columnconfigure(1, weight=4) # Name
        self.scrollable_frame.grid_columnconfigure(2, weight=3) # Program
        self.scrollable_frame.grid_columnconfigure(3, weight=1) # Faces
        self.scrollable_frame.grid_columnconfigure(4, weight=1) # Photo
        self.scrollable_frame.grid_columnconfigure(5, weight=1) # View Card

        # Right side - ID card preview
        self.card_preview_frame = ctk.CTkFrame(frame)
        self.card_preview_frame.grid(row=0, column=1, padx=(10, 20), pady=20, sticky="nsew")
        self.card_preview_frame.grid_propagate(False) 
        self.card_preview_frame.grid_rowconfigure(1, weight=1)
        self.card_preview_frame.grid_columnconfigure(0, weight=1)

        ctk.CTkLabel(self.card_preview_frame, text="ID Card Preview", font=ctk.CTkFont(size=20, weight="bold")).grid(row=0, column=0, pady=(20, 10))
        
        # --- UI IMPROVEMENT ---: Center the preview label
        self.card_display_label = ctk.CTkLabel(self.card_preview_frame, text="Select a student to view their ID card")
        self.card_display_label.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")

        return frame

    def populate_student_list(self):
        # Reload data from file in case it was modified
        self.controller.student_data = self.controller.load_student_database()

        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()

        # Header
        headers = ["ID", "Name", "Program", "Faces", "Photo", "Action"]
        for i, header in enumerate(headers):
            ctk.CTkLabel(self.scrollable_frame, text=header, font=ctk.CTkFont(weight="bold")).grid(row=0, column=i, padx=5, pady=10, sticky="w")

        # Student rows
        if not self.controller.student_data:
            ctk.CTkLabel(self.scrollable_frame, text="No students found in the database.").grid(row=1, column=0, columnspan=6, pady=20)
            return

        for i, (student_id, info) in enumerate(self.controller.student_data.items()):
            row_num = i + 1
            face_folder = self.controller.FACE_DB_PATH / student_id
            face_count = len(list(face_folder.glob("*.jpg"))) if face_folder.exists() else 0
            photo_exists = (self.controller.PHOTO_DIR / f"{student_id}.jpg").exists()

            ctk.CTkLabel(self.scrollable_frame, text=student_id).grid(row=row_num, column=0, padx=5, pady=5, sticky="w")
            ctk.CTkLabel(self.scrollable_frame, text=info.get('name', 'N/A')).grid(row=row_num, column=1, padx=5, pady=5, sticky="w")
            ctk.CTkLabel(self.scrollable_frame, text=info.get('program', 'N/A')).grid(row=row_num, column=2, padx=5, pady=5, sticky="w")
            ctk.CTkLabel(self.scrollable_frame, text=f"{face_count}", text_color="green" if face_count > 0 else "orange").grid(row=row_num, column=3, padx=5, pady=5, sticky="w")
            ctk.CTkLabel(self.scrollable_frame, text="✓ Yes" if photo_exists else "✗ No", text_color="green" if photo_exists else "red").grid(row=row_num, column=4, padx=5, pady=5, sticky="w")
            
            ctk.CTkButton(self.scrollable_frame, text="View Card", width=80, height=28, command=lambda sid=student_id: self.show_student_card(sid)).grid(row=row_num, column=5, padx=5, pady=5)
    
    def show_student_card(self, student_id):
        card_path = self.controller.OUTPUT_DIR / f"{student_id}_id_card.png"
        if not card_path.exists():
            if student_id in self.controller.student_data:
                self.controller.create_id_card(self.controller.student_data[student_id])
            else:
                self.card_display_label.configure(image=None, text=f"Student data for {student_id}\nnot found.")
                return

        if card_path.exists():
            try:
                card_img = Image.open(card_path)

                # --- REFINED LOGIC ---: Cleaner resizing calculation
                # Use the preview frame's actual dimensions, which are now stable.
                frame_w = self.card_preview_frame.winfo_width()
                frame_h = self.card_preview_frame.winfo_height()

                # Add a small padding
                display_w = frame_w - 40 
                display_h = frame_h - 80 # Account for title

                img_aspect = card_img.width / card_img.height
                display_aspect = display_w / display_h
                
                if img_aspect > display_aspect:
                    new_w = display_w
                    new_h = int(new_w / img_aspect)
                else:
                    new_h = display_h
                    new_w = int(new_h * img_aspect)

                card_img = card_img.resize((new_w, new_h), Image.Resampling.LANCZOS)
                
                card_tk = ImageTk.PhotoImage(card_img)
                self.card_display_label.configure(image=card_tk, text="")
                self.card_display_label.image = card_tk
            except Exception as e:
                logger.error(f"Error displaying card for {student_id}: {e}")
                self.card_display_label.configure(image=None, text="Error loading ID card image.")
        else:
            self.card_display_label.configure(image=None, text="Could not generate ID card.")

    def update_next_student_id(self):
        next_id = self.controller.get_next_student_id()
        if hasattr(self, 'next_id_var'):
            self.next_id_var.set(next_id)

    def show_register_frame(self):
        self.stop_video_feed()
        self.list_frame.grid_forget()
        self.register_frame.grid(row=0, column=1, sticky="nsew")
        self.reset_registration()

    def show_list_frame(self):
        self.stop_video_feed()
        self.register_frame.grid_forget()
        self.list_frame.grid(row=0, column=1, sticky="nsew")
        self.populate_student_list()
        self.card_display_label.configure(image=None, text="Select a student to view their ID card")

    def start_video_feed(self):
        if self.cap is None or not self.cap.isOpened():
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.status_label.configure(text="Error: Cannot open camera.", text_color="red")
                self.cap = None
                return
        self.is_capturing = True
        self.update_video_feed()

    def stop_video_feed(self):
        self.is_capturing = False
        # Delay release to avoid race conditions
        self.after(50, self._release_cap)

    def _release_cap(self):
        if self.cap:
            self.cap.release()
            self.cap = None
            
    def update_video_feed(self):
        if not self.is_capturing or not self.cap:
            return
        
        ret, frame = self.cap.read()
        if not ret:
            self.after(10, self.update_video_feed)
            return
        
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.controller.face_cascade.detectMultiScale(gray, 1.2, 5, minSize=(120, 120))
        
        current_time = time.time()
        
        if len(faces) > 0 and (current_time - self.last_capture_time > self.AUTO_CAPTURE_DELAY):
            self.last_capture_time = current_time
            
            if self.capture_mode == 'verification' and self.captured_count < self.VERIFICATION_IMAGE_COUNT:
                x, y, w, h = faces[0]
                face_img = frame[y:y+h, x:x+w]
                face_resized = cv2.resize(face_img, (220, 220))
                filepath = self.controller.FACE_DB_PATH / self.current_student_info['id'] / f"face_{self.captured_count+1:02d}.jpg"
                cv2.imwrite(str(filepath), face_resized)
                
                self.captured_count += 1
                progress = self.captured_count / self.VERIFICATION_IMAGE_COUNT
                self.progress_bar.set(progress)
                self.progress_label.configure(text=f"Progress: {self.captured_count}/{self.VERIFICATION_IMAGE_COUNT} captured")
                
                if self.captured_count >= self.VERIFICATION_IMAGE_COUNT:
                    self.status_label.configure(text="Verification capture complete!", text_color="green")
                    self.after(500, self.proceed_to_photo) # Auto-proceed

            elif self.capture_mode == 'id_photo':
                if len(faces) == 1:
                    x, y, w, h = faces[0]
                    padding = 40
                    x1, y1 = max(0, x - padding), max(0, y - padding - 20)
                    x2, y2 = min(frame.shape[1], x + w + padding), min(frame.shape[0], y + h + padding)
                    photo = frame[y1:y2, x1:x2]
                    photo_path = self.controller.PHOTO_DIR / f"{self.current_student_info['id']}.jpg"
                    cv2.imwrite(str(photo_path), photo)
                    
                    self.status_label.configure(text="ID Photo captured! Finalizing...", text_color="green")
                    self.after(500, self.complete_registration) # Auto-complete
        
        display_text = ""
        rect_color = (0, 255, 0) # Green for OK
        if self.capture_mode == 'verification':
            display_text = f"Capturing verification: {self.captured_count}/{self.VERIFICATION_IMAGE_COUNT}"
        elif self.capture_mode == 'id_photo':
            if len(faces) == 1:
                display_text = "Perfect! Capturing photo..."
            elif len(faces) > 1:
                display_text = "Multiple faces detected. Please show only one."
                rect_color = (0, 0, 255) # Red for error
            else:
                display_text = "Position your face in the frame."
                rect_color = (0, 255, 255) # Yellow for warning

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), rect_color, 2)
        
        if display_text:
            cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        
        # --- UI IMPROVEMENT ---: Resizing logic for the video feed
        frame_w = self.display_frame.winfo_width()
        frame_h = self.display_frame.winfo_height()
        img_aspect = img.width / img.height
        frame_aspect = frame_w / frame_h
        if img_aspect > frame_aspect:
            new_w = frame_w - 20
            new_h = int(new_w / img_aspect)
        else:
            new_h = frame_h - 20
            new_w = int(new_h * img_aspect)

        img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
        
        img_tk = ImageTk.PhotoImage(image=img)
        self.video_label.configure(image=img_tk)
        self.video_label.image = img_tk
        self.after(15, self.update_video_feed)

    def start_verification_capture(self):
        if hasattr(self, "start_verification_btn"):
            self.start_verification_btn.configure(state="disabled")
        self.capture_mode = 'verification'
        self.captured_count = 0
        self.last_capture_time = 0
        self.status_label.configure(text="Auto-capturing verification photos...", text_color=("black", "white"))
        self.start_video_feed()

    def start_id_photo_capture(self):
        self.capture_mode = 'id_photo'
        self.last_capture_time = 0
        self.status_label.configure(text="Auto-capturing ID photo. Please center your face.", text_color=("black", "white"))
        self.start_video_feed()
        
    def on_closing(self):
        self.stop_video_feed()
        self.destroy()

In [ ]:
if __name__ == "__main__":
    controller = IDCardController()
    app = App(controller)
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()